In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import Counter
import datetime
import time
import os
import json

from src.data import NSynth
from src.utils import print_and_log
from src.models import Classifier

/opt/conda/lib/python3.6/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
if torch.cuda.is_available():  
    device = torch.device("cuda:0")
else:  
    device = torch.device("cpu") 

In [3]:
hparams = {
    'checkpoint' : None,
    'target' : 'character',
    'instrument_source' : [0, 1, 2],
    'sample_rate' : 16000,
    'n_samples' : 64000,
    'feature_type' : 'mel',
    'random_crop' : True,
    'resize' : None,
    'normalize' : True,
    'standardize' : True,
    'standardize_mean' : 0.3356,
    'standardize_std' : 0.2212,
    'spec_augment' : True,
    'remove_synth_lead' : True,
    'n_samples_per_class' : None,
    'n_epochs' : 50,
    'batch_size' : 32,
    'lr' : 0.000001,
    'hidden_dim' : 512,
    'display_iters' : 100,
    'val_iters' : 1000,
    'n_val_samples' : 1000, 
    'n_early_stopping' : 5 # stop if validation doesn't improve after this number of validation cycles
}

In [7]:
train_dataset = NSynth(
    'data/nsynth', 
    'train',
    include_meta=True, 
    instrument_source=hparams['instrument_source'], 
    sample_rate=hparams['sample_rate'], 
    n_samples=hparams['n_samples'], 
    feature_type=hparams['feature_type'], 
    random_crop=hparams['random_crop'], 
    resize=hparams['resize'], 
    normalize=hparams['normalize'], 
    standardize=hparams['standardize'], 
    standardize_mean=hparams['standardize_mean'], 
    standardize_std=hparams['standardize_std'], 
    spec_augment=hparams['spec_augment'],
    remove_synth_lead=hparams['remove_synth_lead'], 
    n_samples_per_class=hparams['n_samples_per_class']
)

val_dataset = NSynth(
    'data/nsynth',
    'val',
    include_meta=True, 
    instrument_source=hparams['instrument_source'], 
    sample_rate=hparams['sample_rate'], 
    n_samples=hparams['n_samples'], 
    feature_type=hparams['feature_type'], 
    random_crop=hparams['random_crop'], 
    resize=hparams['resize'], 
    normalize=hparams['normalize'], 
    standardize=hparams['standardize'], 
    standardize_mean=hparams['standardize_mean'], 
    standardize_std=hparams['standardize_std'], 
    remove_synth_lead=hparams['remove_synth_lead']
)

In [25]:
class_ctr = dict(train_dataset.meta[hparams['target']].value_counts())
index_to_name = dict(enumerate(sorted(class_ctr.keys())))
name_to_index = dict([(v, k) for k, v in index_to_name.items()])
class_weights = dict([(k, max(class_ctr.values()) / class_ctr[k]) for k in class_ctr.keys()])

sample_weights = [class_weights[train_dataset.meta.iloc[i][hparams['target']]] for i in range(len(train_dataset))]
sampler = torch.utils.data.sampler.WeightedRandomSampler(sample_weights, len(sample_weights))

In [30]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=hparams['batch_size'], sampler=sampler)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=hparams['batch_size'], shuffle=True)

In [31]:
model = Classifier(n_classes=len(class_ctr), h_dim=hparams['hidden_dim']).to(device)
print('# of parameters : {}'.format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

# of parameters : 2328258


In [32]:
optimizer = torch.optim.Adam(model.parameters(), lr=hparams['lr'])

In [33]:
timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d-%H%M%S')
results_dir = 'train_results/classifier/{}'.format(timestamp)
os.makedirs(results_dir)

with open(os.path.join(results_dir, 'hparams.json'), 'w') as fp:
    json.dump(hparams, fp)
    
with open(os.path.join(results_dir, 'index_to_name.json'), 'w') as fp:
    json.dump(index_to_name, fp)

log_file = os.path.join(results_dir, 'train_log.txt')
log = open(log_file, 'w')
log.close()
print_and_log('{} {}'.format(train_dataset.__class__.__name__, model.__class__.__name__), log_file)

for k, v in hparams.items(): print_and_log('{} : {}'.format(k, v), log_file)

NSynth Classifier
checkpoint : None
target : character
instrument_source : [0, 1, 2]
sample_rate : 16000
n_samples : 64000
feature_type : mel
random_crop : True
resize : None
normalize : True
standardize : True
standardize_mean : 0.3356
standardize_std : 0.2212
spec_augment : True
remove_synth_lead : True
n_samples_per_class : None
n_epochs : 50
batch_size : 32
lr : 1e-06
hidden_dim : 512
display_iters : 100
val_iters : 1000
n_val_samples : 1000
n_early_stopping : 5


In [ ]:
loss_fn = nn.CrossEntropyLoss()

ckpt_weights_path = None
best_loss = 1e10
since_best = 0
done = False

if hparams['checkpoint']:
    print_and_log('Resuming training from {}'.format(hparams['checkpoint']), log_file)
    ckpt = torch.load(hparams['checkpoint'])
    epoch = ckpt['epoch']
    itr = ckpt['itr']
    optimizer.load_state_dict(ckpt['optimizer'])
    model.load_state_dict(ckpt['model'])
    best_loss = ckpt['best_loss']
else:
    epoch = 0
    itr = 0

for epoch in range(epoch, hparams['n_epochs']):
    if done:
        break
        
    for batch in train_dataloader:
        if done:
            break
            
        itr += 1
        features = batch[0].to(device)
        labels = torch.tensor([name_to_index[x] for x in batch[1][hparams['target']]], 
                              dtype=torch.long, device=device)
        outputs = model(features)
        
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (itr % hparams['display_iters'] == 0) or (itr == 1):
            acc = (outputs.argmax(-1) == labels).float().mean()
            print_and_log('[{}, {:5d}] loss : {:.4f}, acc : {:.4f}'.format(epoch, itr, loss.item(), acc.item()), log_file)
            
    model.eval()
    with torch.no_grad():
        ct, i, val_loss, val_acc = 0, 0, 0.0, 0.0
        for batch in val_dataloader:
            i += 1
            ct += batch[0].size(0)
            features = batch[0].to(device)
            labels = torch.tensor([name_to_index[x] for x in batch[1][hparams['target']]], 
                                  dtype=torch.long, device=device)
            outputs = model(features)

            loss = loss_fn(outputs, labels)
            acc = (outputs.argmax(-1) == labels).float().mean()

            val_loss += (loss.item() - val_loss)/i
            val_acc += (acc.item() - val_acc)/i

            if ct >= hparams['n_val_samples']:
                break

    print_and_log('Val - loss : {:.4f}, acc : {:.4f}'.format(val_loss, val_acc), log_file)

    if val_loss < best_loss:
        since_best = 0
        best_loss = val_loss

        # save weights
        ckpt_weights_path = os.path.join(results_dir, 'model-{}.weights'.format(itr))
        torch.save(model.state_dict(), ckpt_weights_path)
        print_and_log('Weights saved in {}'.format(ckpt_weights_path), log_file)

        # save meta information
        ckpt_meta_path = os.path.join(results_dir, 'checkpoint')
        torch.save({
            'best_loss' : best_loss,
            'epoch' : epoch,
            'itr' : itr,
            'optimizer' : optimizer.state_dict(),
            'model' : model.state_dict()
        }, ckpt_meta_path)
    else:
        since_best += 1
        if since_best >= hparams['n_early_stopping']:
            done = True
            print_and_log('Early stopping... training complete', log_file)

    model.train()

[0,     1] loss : 0.6431, acc : 0.6250
[0,   100] loss : 0.5987, acc : 0.7188
[0,   200] loss : 0.6358, acc : 0.5312
[0,   300] loss : 0.6538, acc : 0.6250
[0,   400] loss : 0.6399, acc : 0.6562
[0,   500] loss : 0.6381, acc : 0.5938
[0,   600] loss : 0.6003, acc : 0.7500
[0,   700] loss : 0.6336, acc : 0.5312
[0,   800] loss : 0.5980, acc : 0.6875
[0,   900] loss : 0.6550, acc : 0.5312
[0,  1000] loss : 0.5510, acc : 0.7500
[0,  1100] loss : 0.5641, acc : 0.6562
[0,  1200] loss : 0.5430, acc : 0.7812
[0,  1300] loss : 0.5795, acc : 0.7188
[0,  1400] loss : 0.5871, acc : 0.7500
[0,  1500] loss : 0.4022, acc : 0.8438
[0,  1600] loss : 0.4564, acc : 0.8125
[0,  1700] loss : 0.5190, acc : 0.7500
[0,  1800] loss : 0.5323, acc : 0.7500
[0,  1900] loss : 0.5186, acc : 0.7500
[0,  2000] loss : 0.6010, acc : 0.6875
[0,  2100] loss : 0.5448, acc : 0.7188
[0,  2200] loss : 0.4454, acc : 0.8438
[0,  2300] loss : 0.5824, acc : 0.6875
[0,  2400] loss : 0.5247, acc : 0.7188
[0,  2500] loss : 0.4421,